# Population

In [ ]:
from pathlib import Path

import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd
from configobj import ConfigObj
from folium.plugins import HeatMap
from helpers import configreader as cfr
from helpers import osmhelper as oh
from helpers import ready4robots as r4r

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
csv10m = Path((Path(".").parent), "csvs", "Freiberg, Sachsen-10m.zip")
df = pd.read_csv(csv10m, compression='zip')
df.columns = ["id", "lon", "lat", "pop"]
df = df.set_index("id")
df

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
ax = df.plot.hexbin(x="lon",
                    y="lat",
                    C="pop",
                    reduce_C_function=np.sum,
                    gridsize=50,
                    ax=ax)

In [ ]:
tags = {"type": "boundary"}
place = "Freiberg, Sachsen"
# gdf = ox.geometries_from_place(, tags)
# gdf.plot()
bbox = ox.geocoder.geocode_to_gdf(place, buffer_dist=200)
geom = bbox.geometry.loc[0]

p = gpd.GeoSeries(geom).geometry.boundary
ax = p.plot(color=None, edgecolor='k', linewidth=2, ax=ax)
figure = ax.get_figure()
figure

In [ ]:
def heat_map(df):
    # lat/long/weight

    try:
        data = df[["lat", "lon", "pop"]]
    except KeyError:
        data = df[["x", "y", "einwohner"]]

    # print(df["2"].max())

    m = folium.Map([50.9, 13.37], zoom_start=12)

    HeatMap(data, min_opacity=0.4,
            gradient={.4: 'blue', .8: 'lime',
                      0.9: 'yellow', 0.95: 'orange', 1: 'red'}
            ).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
    folium.LayerControl().add_to(m)

    # m.save('{0}.html'.format('Freiberg-census-10m'))
    return m

In [ ]:
heat_map(df)

In [ ]:
cfg = ConfigObj("notebook.ini")
scenarios = cfr.get_scenarios(cfg)
G = oh.get_place(placename=place, simplify=True,
                 clean_periphery=True,
                 file_identifier="", retain_all=True)

In [ ]:
G_nx = r4r.load_pop_scenario(cfg, G, place, "Optimal")
to_remove = []
for node in G_nx.nodes():
    try:
        G_nx.nodes[node]["pop"]
    except KeyError:
        to_remove.append(node)

G.remove_nodes_from(to_remove)
x = ox.plot_graph(G,
                  figsize=(16, 12),
                  node_color=ox.plot.get_node_colors_by_attr(G, "pop"),
                  node_size=5,
                  edge_linewidth=0.1)
x